In [1]:
import pandas as pd
import numpy as np

In [2]:
FINDER=pd.read_csv(r'block_finder_key_gene.csv')

In [3]:
GAPR=pd.read_excel(r'test_BLOCK_GAPR_AP.xlsx')

In [4]:
FINDER

,gene_id,renum,From,Species,Gene Name,Score
0,351,161,APP,Homo sapiens,amyloid beta precursor protein(APP),9.919679e-01
1,3172,1243,HNF4A,Homo sapiens,hepatocyte nuclear factor 4 alpha(HNF4A),9.688534e-01
2,6872,2143,TAF1,Homo sapiens,TATA-box binding protein associated factor 1(T...,9.405395e-01
3,2885,1148,GRB2,Homo sapiens,growth factor receptor bound protein 2(GRB2),9.234427e-01
4,4149,1419,MAX,Homo sapiens,MYC associated factor X(MAX),9.164938e-01
...,...,...,...,...,...,...
1197,30009,1633,TBX21,Homo sapiens,T-box transcription factor 21(TBX21),6.500000e-07
1198,6351,1863,CCL4,Homo sapiens,C-C motif chemokine ligand 4(CCL4),6.500000e-07
1199,3098,1218,HK1,Homo sapiens,hexokinase 1(HK1),6.500000e-07
1200,59339,774,PLEKHA2,Homo sapiens,pleckstrin homology domain containing A2(PLEKHA2),6.500000e-07


In [5]:
GAPR

,AP,layer,connected component changes,LCC changes
0,125,1,1,0.000808
1,240,1,1,0.000808
2,801,1,4,0.002020
3,1741,1,2,0.001616
4,3480,1,2,0.001212
...,...,...,...,...
1105,29102,13,2,0.266667
1106,57169,13,1,0.133333
1107,9968,13,1,0.133333
1108,3818,13,1,0.000000


In [7]:
pd.merge(FINDER,GAPR,left_on='gene_id',right_on='AP').to_csv(r'block_finder_key_gene_layer.csv',index=False)

In [98]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter, LogLocator


# 假设您已经读取了数据，并定义了 finder_data 和 layers
# 如果还没有，请取消以下注释并替换为您的数据加载代码
finder_data = pd.read_csv('block_finder_key_gene_layer.csv')
layers = sorted(finder_data['layer'].unique())
# 获取层的列表，并按顺序排序

# 定义颜色列表
colors = ['skyblue', 'lightgreen', 'lightcoral', 'plum', 'khaki', 'lightsalmon', 'lightpink',
          '#9c88ff', '#ff6b6b', '#feca57', '#C4E538', '#1dd1a1', '#48dbfb', '#54a0ff', '#00d2d3']

# 创建图形和坐标轴
fig, ax = plt.subplots(figsize=(6, 8))

# 准备数据
plot_data = [finder_data[finder_data['layer'] == layer]['LCC changes'].dropna() for layer in layers]

# 处理 LCC changes 数据中的零或负值，替换为很小的正数
for i in range(len(plot_data)):
    plot_data[i] = plot_data[i].replace(0, 1e-3)
    plot_data[i] = plot_data[i].clip(lower=1e-3)

# 绘制小提琴图
violin = ax.violinplot(plot_data, positions=np.arange(len(layers)) + 1, widths=0.7,
                       showextrema=False, vert=False)

# 绘制箱线图
bplot = ax.boxplot(plot_data, positions=np.arange(len(layers)) + 1, widths=0.15,
                   patch_artist=True, whis=1.5, vert=False,
                   flierprops={'marker': '.', 'markeredgecolor': '#353b48', 'markersize': 2},
                   medianprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5},
                   whiskerprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5},
                   capprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5})

# 设置箱线图的颜色
for patch, color in zip(bplot['boxes'], colors[:len(bplot['boxes'])]):
    patch.set_facecolor(color)
    patch.set_edgecolor('#353b48')
    patch.set_linewidth(0.5)

# 设置小提琴图的颜色
for body, color in zip(violin['bodies'], colors[:len(violin['bodies'])]):
    body.set_facecolor(color)
    body.set_edgecolor('black')
    body.set_linewidth(0.5)
    body.set_alpha(0.5)


plt.tick_params(top=False,bottom=True,left=False,right=True)
# 设置轴标签和标题
ax.set_yticks(np.arange(len(layers)) + 1)
ax.set_yticklabels(layers, rotation=0, fontsize=12)
ax.set_xlabel('LCC Changes', fontsize=14)
ax.set_title('LCC Changes', fontsize=16)
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(0.5)
# 设置 x 轴为对数尺度，并限制范围，增加第一个刻度和 Y 轴的距离
ax.set_xscale('log')
ax.set_xlim(7*1e-4,0.5)  # 将左边界从 0.4 调整为 0.5，增加间距

# 反转 x 轴，使图从右往左绘制
ax.invert_xaxis()

# 格式化 x 轴刻度标签
#ax.xaxis.set_major_locator(LogLocator(base=10.0, numticks=4))#
def log_formatter(x, pos):
    if x >= 1:
        return f'{x:.0f}'
    else:
        return f'{x:.0e}'
ax.xaxis.set_major_formatter(FuncFormatter(log_formatter))

# 移除网格线
ax.grid(False)

# 显示 X 和 Y 轴的刻度线，并设置刻度标签的字体大小
# plt.tick_params(axis='both',direction="out")

# 调整布局
plt.tight_layout()

# 显示图形
# plt.show()
plt.savefig(r'LCC method.png', transparent=True,dpi=600)
plt.close('all')

In [99]:
fig, ax = plt.subplots(figsize=(6, 8))
plot_data = [finder_data[finder_data['layer'] == layer]['Score'].dropna() for layer in layers]

# 绘制小提琴图
violin = ax.violinplot(plot_data, positions=np.arange(len(layers)) + 1, widths=0.7, showextrema=False, vert=False)

# 绘制箱线图
bplot = ax.boxplot(plot_data, positions=np.arange(len(layers)) + 1, widths=0.15, patch_artist=True, whis=1.5, vert=False,
                   flierprops={'marker': '.', 'markeredgecolor': '#353b48', 'markersize': 2},
                   medianprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5},
                   whiskerprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5},
                   capprops={'linestyle': '-', 'color': '#353b48', 'linewidth': 0.5})

# 设置箱线图的颜色
for patch, color in zip(bplot['boxes'], colors[:len(bplot['boxes'])]):
    patch.set_facecolor(color)
    patch.set_edgecolor('#353b48')
    patch.set_linewidth(0.5)

# 设置小提琴图的颜色
for body, color in zip(violin['bodies'], colors[:len(violin['bodies'])]):
    body.set_facecolor(color)
    body.set_edgecolor('black')
    body.set_linewidth(0.5)
    body.set_alpha(0.5)

# 设置轴标签和标题
for label in ax.get_yticklabels():
    label.set_horizontalalignment('center')
    label.set_x(-0.01)  # 负值将标签向左移动，调整此值以避免重叠
ax.set_yticks(np.arange(len(layers)) + 1)
ax.set_yticklabels(layers, rotation=0, fontsize=12)
ax.set_xlabel('Finder Score', fontsize=14)
ax.set_title('各层 Finder Score 的小提琴图和箱线图', fontsize=16)
plt.tick_params(top=False,bottom=True,left=True,right=False)
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(0.5)
# 添加网格线
ax.grid(False)
ax.tick_params(axis='both')
# 调整布局
plt.tight_layout()
# plt.show()
plt.savefig(r'Finder method.png', transparent=True,dpi=600)
plt.close('all')

C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 21508 (\N{CJK UNIFIED IDEOGRAPH-5404}) missing from current font.
  plt.tight_layout()
C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 23618 (\N{CJK UNIFIED IDEOGRAPH-5C42}) missing from current font.
  plt.tight_layout()
C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 30340 (\N{CJK UNIFIED IDEOGRAPH-7684}) missing from current font.
  plt.tight_layout()
C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 23567 (\N{CJK UNIFIED IDEOGRAPH-5C0F}) missing from current font.
  plt.tight_layout()
C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 25552 (\N{CJK UNIFIED IDEOGRAPH-63D0}) missing from current font.
  plt.tight_layout()
C:\Users\11340\AppData\Local\Temp\ipykernel_8812\2737985638.py:43: UserWarning: Glyph 29748 (\N{CJK UNIFIED IDEOGRAPH-7434}) missing fr

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np

# 读取数据
data = pd.read_csv(r'big_block_key_all_final.csv')

# 初始化 Min-Max 标准化器
min_max_scaler = MinMaxScaler()
columns_to_standardize = ['Similarity', 'Finder_Score', 'Score_total', 'LCC changes']

# 标准化指定的列
data[columns_to_standardize] = min_max_scaler.fit_transform(data[columns_to_standardize])

# 定义直方图的分箱
bins = np.arange(0.05, 1.05, 0.05)

# 计算每个标准化得分的直方图频数
hist_similarity, _ = np.histogram(data['Similarity'], bins=bins)
hist_finder_score, _ = np.histogram(data['Finder_Score'], bins=bins)
hist_lcc_changes, _ = np.histogram(data['LCC changes'], bins=bins)
hist_score_total, _ = np.histogram(data['Score_total'], bins=bins)

# 将频数堆叠成二维数组
hist_counts = np.vstack([hist_similarity, hist_finder_score, hist_lcc_changes, hist_score_total])

# 计算每个分箱的总频数
total_counts = hist_counts.sum(axis=0)

# 防止除以零
total_counts = np.where(total_counts == 0, 1, total_counts)

# 将频数转换为百分比
hist_percentages = hist_counts / total_counts * 100

# 定义分箱的中心位置
bin_centers = (bins[:-1] + bins[1:]) / 2

# 创建堆叠的百分比柱状图
fig, ax = plt.subplots(figsize=(8, 8))

labels = ['Similarity', 'FINDER Score', 'LCC Changes', 'NASH Score']
colors = ['#f2b56f', '#f57c6e', '#84c3b7', '#b8aeed']

bottom = np.zeros(len(bin_centers))

for i in range(len(labels)):
    ax.bar(bin_centers, hist_percentages[i], width=0.045, bottom=bottom, edgecolor='black', label=labels[i], color=colors[i], align='center')
    bottom += hist_percentages[i]

# 添加标签和标题
ax.set_xlabel('Standardized Scores', fontsize=20)
ax.set_ylabel('Percentage (%)', fontsize=20)


# 设置坐标轴范围
ax.set_xlim(0.1, 1)
ax.set_ylim(0, 100)

# 设置坐标轴刻度字体大小
ax.tick_params(axis='both', which='major', labelsize=12)

# 设置坐标轴颜色
ax.spines['top'].set_color('black')
ax.spines['right'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')

# 移除网格线
ax.grid(False)

# 添加图例
ax.legend(prop={'family': 'Arial', 'size': 20})
plt.tick_params(top=False,bottom=True,left=True,right=False)
# 显示图形
plt.tight_layout()
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(0.5)
plt.xticks(fontname='Arial', fontsize=20)
plt.yticks(fontname='Arial', fontsize=20)
# plt.show()
plt.savefig(r'Combination_Scores_new.svg', transparent=True, dpi=600)
plt.close('all')

In [2]:
import pandas as pd

In [22]:
nature_infor=pd.read_csv(r'compound_information.csv')

In [26]:
nature_results=pd.read_table(r'results.txt',sep='\t',header=None)

In [27]:
nature_results_ROR=nature_results[nature_results[3]=='P51449']

In [29]:


# Standardize the 'Drug_num' column: strip whitespaces and normalize case
nature_results_ROR[1] = nature_results_ROR[1].str.strip().str.upper()
nature_infor['Drug_num'] = nature_infor['Drug_num'].str.strip().str.upper()

C:\Users\11340\AppData\Local\Temp\ipykernel_64168\4271807795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nature_results_ROR[1] = nature_results_ROR[1].str.strip().str.upper()


In [31]:
pd.merge(nature_results_ROR,nature_infor,left_on=1,right_on='Drug_num').to_csv(r'D:\文章\RORgt\DTI_prediction\wSDTNBI原始数据\results_ROR.csv',index=False)